**Obesity research in specialty journals from 2000 to 2023: A bibliometric analysis**

In [ ]:
import os
import pandas as pd
import glob
import re
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.flush_and_unmount()  # Unmount Google Drive if already mounted
drive.mount('/content/drive')

Mounted at /content/drive


**Merging the CSV files**

In [ ]:
# Set working directory
folder = '/content/drive/My Drive/OBESITY.JOURNALS/CSV'
os.chdir(folder)

In [ ]:
# 1. List all CSV files
csv_files = glob.glob(os.path.join(folder, '*.csv'))

# 2. Group files by ISSN
issn_pattern = re.compile(r'results_(\d{8})_id_.*\.csv')
issn_groups = {}
for file in csv_files:
    match = issn_pattern.search(os.path.basename(file))
    if match:
        issn = match.group(1)
        issn_groups.setdefault(issn, []).append(file)

# 3. Merge files for each ISSN
issn_dfs = []
for issn, files in issn_groups.items():
    dfs = []
    for f in files:
        df = pd.read_csv(f)
        dfs.append(df)
    # Merge all DataFrames for this ISSN on 'id'
    merged = dfs[0]
    for df in dfs[1:]:
        # Avoid duplicate columns
        merged = pd.merge(merged, df, on='id', how='outer', suffixes=('', '_dup'))
        # Drop duplicate columns
        dup_cols = [col for col in merged.columns if col.endswith('_dup')]
        merged = merged.drop(columns=dup_cols)
    # Optionally, add ISSN as a column
    merged['issn'] = issn
    issn_dfs.append(merged)

# 4. Merge all ISSN DataFrames on 'id'
final_df = pd.concat(issn_dfs, ignore_index=True, sort=False)

# 5. Export as CSV
final_df.to_csv(os.path.join(folder, 'merged_results.csv'), index=False)
print("Merging complete! Files saved as 'merged_results.csv'")

Merging complete! Files saved as 'merged_results.csv'


**Filtering the dataset**

In [ ]:
# Load the dataset as df
filename = os.path.join(folder, 'merged_results.csv')
df = pd.read_csv(filename)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44197 entries, 0 to 44196
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    44197 non-null  object 
 1   funders               18315 non-null  object 
 2   abstract              35729 non-null  object 
 3   category_bra          30654 non-null  object 
 4   category_for          43144 non-null  object 
 5   category_hra          28490 non-null  object 
 6   category_hrcs_hc      26377 non-null  object 
 7   category_rcdc         37194 non-null  object 
 8   category_sdg          7393 non-null   object 
 9   category_uoa          38048 non-null  object 
 10  category_hrcs_rac     17141 non-null  object 
 11  category_icrp_cso     4368 non-null   object 
 12  category_icrp_ct      6665 non-null   object 
 13  recent_citations      44167 non-null  float64
 14  reference_ids         36282 non-null  object 
 15  concepts           

In [ ]:
print(df['type'].unique())

['article' nan]


In [ ]:
nan_type = df['type'].isna().sum()
print(f"Number of NaNs in 'type' column: {nan_type}")

Number of NaNs in 'type' column: 30


In [ ]:
df['journal.title'].value_counts()

,count
journal.title,
Obesity Surgery,11117
International Journal of Obesity,10077
Obesity,8873
Obesity Research & Clinical Practice,2867
Obesity Reviews,2820
Pediatric Obesity,1715
Childhood Obesity,1700
Obesity Facts,1202
Journal of Obesity,821


In [ ]:
# Create a copy to avoid modifying the original dataframe
filtered_df = df.copy()

# Convert 'year' to numeric, but keep original values where conversion fails
filtered_df['year_numeric'] = pd.to_numeric(filtered_df['year'], errors='coerce')

# Create filter condition:
# Keep rows where year is NOT numeric (string values) OR where numeric year is between 2000-2023
condition = (
    filtered_df['year_numeric'].isna() |  # Keep rows where conversion to numeric failed (strings)
    ((filtered_df['year_numeric'] >= 2000) & (filtered_df['year_numeric'] <= 2023))  # Keep numeric years 2000-2023
)

# Apply the filter
filtered_df = filtered_df[condition]

# Drop the helper column if you don't need it
filtered_df = filtered_df.drop('year_numeric', axis=1)

In [ ]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36918 entries, 0 to 44196
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    36918 non-null  object 
 1   funders               16390 non-null  object 
 2   abstract              30061 non-null  object 
 3   category_bra          25944 non-null  object 
 4   category_for          36075 non-null  object 
 5   category_hra          24208 non-null  object 
 6   category_hrcs_hc      22400 non-null  object 
 7   category_rcdc         31302 non-null  object 
 8   category_sdg          6258 non-null   object 
 9   category_uoa          31936 non-null  object 
 10  category_hrcs_rac     14750 non-null  object 
 11  category_icrp_cso     3847 non-null   object 
 12  category_icrp_ct      5749 non-null   object 
 13  recent_citations      36888 non-null  float64
 14  reference_ids         32756 non-null  object 
 15  concepts              35

In [ ]:
filtered_df = filtered_df[filtered_df['abstract'] != 'nan']

In [ ]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30061 entries, 0 to 43420
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    30061 non-null  object 
 1   funders               15962 non-null  object 
 2   abstract              30061 non-null  object 
 3   category_bra          24511 non-null  object 
 4   category_for          30057 non-null  object 
 5   category_hra          23646 non-null  object 
 6   category_hrcs_hc      21569 non-null  object 
 7   category_rcdc         29610 non-null  object 
 8   category_sdg          5844 non-null   object 
 9   category_uoa          30042 non-null  object 
 10  category_hrcs_rac     14285 non-null  object 
 11  category_icrp_cso     3628 non-null   object 
 12  category_icrp_ct      5293 non-null   object 
 13  recent_citations      30061 non-null  float64
 14  reference_ids         29562 non-null  object 
 15  concepts              30

In [ ]:
filtered_df['journal.title'].value_counts()

,count
journal.title,
Obesity Surgery,7585
Obesity,6854
International Journal of Obesity,6021
Obesity Reviews,2146
Pediatric Obesity,1483
Obesity Research & Clinical Practice,1075
Obesity Facts,880
Childhood Obesity,818
Journal of Obesity,772


In [ ]:
# Export as CSV
filtered_df.to_csv(os.path.join(folder, 'merged_results_filtered.csv'), index=False)
print("Merging complete! Files saved as 'merged_results_filtered.csv'")

Merging complete! Files saved as 'merged_results_filtered.csv'
